In [7]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

In [8]:
path = './'
folders = ['folder_1' ,'folder_2']

In [9]:
x_train = []
y_train = []

x_shape = 256
y_shape = 256
channels = 3

In [10]:
# enumeramos las distintas carpetas
for i_label, folder in enumerate(folders):
    print(folder)
    image_list = os.listdir(folder) #devuelve una lista de los elementos en la carpeta sample_data
    #filtrado, por si hubiera otra cosa que no son imágenes
    image_list = list(filter(lambda x: x.endswith('.jpg'), image_list))
    for img in image_list:
        image_path = os.path.join(folder, img)
        print(image_path)
        im = Image.open(image_path)
        # es importante que todas las imagenes quetan el mismo tamaño
        im = im.resize((x_shape, y_shape), Image.ANTIALIAS)  
        x_train.append(np.asarray(im))
        y_train.append(i_label)

# lo convertimos en arrays de numpy
x_train = np.array(x_train)
y_train = np.array(y_train)

folder_1


FileNotFoundError: [Errno 2] No such file or directory: 'folder_1'

In [11]:
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

In [15]:
def create_model(n_classes=2, show_summary=True):

    base_model = MobileNet(weights='imagenet',include_top=False) #imports the mobilenet model and discards the last 1000 neuron layer.

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024,activation='relu')(x)  #we add dense layers so that the model can learn more complex functions and classify for better results.
    x = Dense(1024,activation='relu')(x)  #dense layer 2
    x = Dense(512,activation='relu')(x)   #dense layer 3

    preds = Dense(n_classes,activation='softmax')(x) #final layer with softmax activation
        
    model = Model(inputs=base_model.input,outputs=preds)
    
    if show_summary: model.summary()

    model.compile(optimizer = 'Adam',
              loss ='categorical_crossentropy',
              metrics = ['accuracy'])
    
    return model


In [16]:
n_classes = 2
model = create_model(n_classes=n_classes)

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, None, None, 3)     0         
_________________________________________________________________
conv1 (Conv2D)               (None, None, None, 32)    864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, None, None, 32)    128       
_________________________________________________________________
conv1_relu (ReLU)            (None, None, None, 32)    0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, None, None, 32)    288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, None, None, 32)    128 

In [ ]:
# pasar las Y a categórico
from keras.utils.np_utils import to_categorical
y_Hot = to_categorical(y_train, num_classes = n_classes)

In [ ]:
# train/test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)

In [ ]:
# entrenar el modelo
model.fit(x_train, y_train, validation_data=[x_test, y_test], epochs = 10)

In [ ]:
# predicciones
model.predict(IMG)